간단한 이론은 ppt 참고 

### 4.5.2. High-Dimensional Linear Regression 

In [1]:
%matplotlib inline
import torch
from torch import nn
from d2l import torch as d2l

We choose our label to be a linear function of our inputs, corrupted by Gaussian noise with zero mean and standard deviation 0.01. To make the effects of overfitting pronounced, we can increase the dimensionality of our problem to d=200 and work with a small training set containing only 20 examples.

-> overfitting 되어 있는 다항식 만들어주기 

In [2]:
def synthetic_data(w, b, num_examples):
    """Generate y = Xw + b + noise"""
    X = d2l.normal(0, 1, (num_examples, len(w)))
    y = d2l.matmul(X, w) + b
    y += d2l.normal(0, 0.01, y.shape) # = noise 에 해당 
    return X, d2l.reshape(y, (-1, 1))


n_train, n_test, num_inputs, batch_size = 20, 100, 200, 5
true_w, true_b = torch.ones((num_inputs, 1)) * 0.01, 0.05

train_data = d2l.synthetic_data(true_w, true_b, n_train)
train_iter = d2l.load_array(train_data, batch_size) # dataloader 
test_data = d2l.synthetic_data(true_w, true_b, n_test)
test_iter = d2l.load_array(test_data, batch_size, is_train=False)

In [25]:
# Initializing model parameters 
# 이는 알고리즘은 학습 중인 데이터의 입력에서 출력까지 특정 매핑 기능에 좋은 weight 집단을 찾기 위해서 임의성을 사용해서 진행하기 때문 
# 하는 이유는 학습을 위해 확률적 경사 하강법을 사용하는데 이 확률적 최적화 알고리즘은 탐색의 시작점을 선택하고 탐색을 진행할 때 임의성을 사용 
# 그래서 네트워크의 가중치가 작은 임의값 (0-1에서와 같이 0에 가까운)으로 초기화되어야 됨. 
def init_params():
    w = torch.normal(0, 1, size=(num_inputs, 1), requires_grad=True)
    b = torch.zeros(1, requires_grad=True)
    return [w, b]

# define L2 norm penalty 
def l2_penalty(w):
    return torch.sum(w.pow(2)) / 2

# train loop 
def train(lambd):
    w, b = init_params()
    net = lambda X: d2l.linreg(X, w, b) # d2l.matmul(X, w) + b
    loss = d2l.squared_loss # (y_hat - d2l.reshape(y, y_hat.shape)) ** 2 / 2
    num_epochs = 100 
    lr = 0.003

    for epoch in range(num_epochs):
        for X, y in train_iter:
            l = loss(net(X), y) + lambd * l2_penalty(w)
            l.sum().backward()
            d2l.sgd([w, b], lr, batch_size)

    print('L2 norm of w:', torch.norm(w).item())

In [28]:
train(lambd=0)

L2 norm of w: 13.211760520935059


In [30]:
train(lambd=3)

L2 norm of w: 0.3699600100517273


### 4.5.4. Concise Implementation 

In [31]:
def train_concise(wd):
    net = nn.Sequential(nn.Linear(num_inputs, 1))
    for param in net.parameters():
        param.data.normal_()
    loss = nn.MSELoss(reduction='none')
    num_epochs, lr = 100, 0.003
    # The bias parameter has not decayed
    trainer = torch.optim.SGD([
        {"params":net[0].weight,'weight_decay': wd},
        {"params":net[0].bias}], lr=lr)

    for epoch in range(num_epochs):
        for X, y in train_iter:
            trainer.zero_grad()
            l = loss(net(X), y)
            l.mean().backward()
            trainer.step()
    print('L2 norm of w:', net[0].weight.norm().item())

train_concise(0)

L2 norm of w: 14.493910789489746


In [32]:
train_concise(3)

L2 norm of w: 0.5018966794013977
